In [1]:
import time
import json
import pandas as pd
from web3 import Web3
import httpx

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
# pd.set_option('display.float_format', lambda x: '%.18f' % x)

thegraph_url = "https://api.thegraph.com/subgraphs/name/proy24/quickswap-polygon"
print('started...')

def get_uniswap_graphql(payload) -> pd.DataFrame:
    try:
        r = httpx.post(
            thegraph_url,
            data=json.dumps(payload),
            timeout=3600
        ).json()
        if r.get('data'):
            return pd.DataFrame(r['data']['pairs'])
        else:
            raise ValueError('shitty service gonna shit the bed again')
    except Exception as e:
        print(f"doh!: {e}")

graphql_payload_1st1000 = {
    "operationName":"pairs",
    "variables":{},
    "query":"query pairs {\n pairs(first: 1000) {\n id token0 {\n symbol\n name\n id\n decimals\n \n}\n token1 {\n symbol\n name\n id\n decimals\n \n}\n reserve0 reserve1 token0Price token1Price reserveUSD reserveETH trackedReserveETH volumeUSD untrackedVolumeUSD txCount \n} \n}\n"
}
graphql_payload_2nd1000 = {
    "operationName":"pairs",
    "variables":{},
    "query":"query pairs {\n pairs(first: 1000, skip:1000) {\n id token0 {\n symbol\n name\n id\n decimals\n \n}\n token1 {\n symbol\n name\n id\n decimals\n \n}\n reserve0 reserve1 token0Price token1Price reserveUSD reserveETH trackedReserveETH volumeUSD untrackedVolumeUSD txCount \n} \n}\n"
}

graphql_payload_3rd1000 = {
    "operationName":"pairs",
    "variables":{},
    "query":"query pairs {\n pairs(first: 1000, skip:2000) {\n id token0 {\n symbol\n name\n id\n decimals\n \n}\n token1 {\n symbol\n name\n id\n decimals\n \n}\n reserve0 reserve1 token0Price token1Price reserveUSD reserveETH trackedReserveETH volumeUSD untrackedVolumeUSD txCount \n} \n}\n"
}


df1 = get_uniswap_graphql(graphql_payload_1st1000)
assert len(df1.index) > 0
df2 = get_uniswap_graphql(graphql_payload_2nd1000)
assert len(df2.index) > 0
df3 = get_uniswap_graphql(graphql_payload_3rd1000)
assert len(df3.index) > 0
df = pd.concat([df1, df2, df3])
print(f"df: {len(df)}")
df['symbols'] = [f"{x['symbol']}/{y['symbol']}" for x, y in zip(df['token0'], df['token1'])]
df['name0'] = [x['name'] for x in df['token0']]
df['name1'] = [x['name'] for x in df['token1']]
df['decimal0'] = [x['decimals'] for x in df['token0']]
df['decimal1'] = [x['decimals'] for x in df['token1']]
df['address0'] = [x['id'] for x in df['token0']]
df['address1'] = [x['id'] for x in df['token1']]

mung_cols = ['reserve0', 'reserve1', 'reserveETH', 'trackedReserveETH', 'volumeUSD', 'untrackedVolumeUSD', \
             'txCount', 'reserveUSD', 'token0Price', 'token1Price', 'decimal0', 'decimal1']
df[mung_cols] = df[mung_cols].apply(pd.to_numeric, errors='coerce')
df = df.rename(columns={'id':'pairAddr', 'token0Price': 'price0', 'token1Price': 'price1'})
df = df[['symbols', 'pairAddr', 'price0', 'price1', 'reserve0', 'reserve1', 'reserveETH', 'reserveUSD',\
         'trackedReserveETH', 'txCount', 'untrackedVolumeUSD', 'volumeUSD', 'decimal0', 'decimal1', 'address0', 'address1']]
df = df.sort_values('reserveUSD',  ascending=False)
df = df.reset_index()
del df['index']
df = df.copy()
#assert df.isnull().values.any()
df.head(10)

started...
df: 3000


,symbols,pairAddr,price0,price1,reserve0,reserve1,reserveETH,reserveUSD,trackedReserveETH,txCount,untrackedVolumeUSD,volumeUSD,decimal0,decimal1,address0,address1
0,USDC/miMATIC,0x160532d2536175d65c03b97b0630a9802c274dad,0.986970,1.013202,8.424094e+06,8.535308e+06,4116.164213,1.684819e+07,4116.164213,800190,4.564450e+08,4.564058e+08,6,18,0x2791bca1f2de4661ed88a30c99a7a9449aa84174,0xa3fa99a148fa48d14ed51d610c367c61876997f1
1,WMATIC/QUICK,0x019ba0325f1988213d448b3472fa1cf8d07618d7,257.947581,0.003877,4.639815e+06,1.798743e+04,3620.361963,1.481882e+07,3620.361963,3995045,1.133434e+09,1.133434e+09,18,18,0x0d500b1d8e8ef31e21c99d1db9a6444d3adf1270,0x831753dd7087cac61ab5644b308642cc1c33dc13
2,WETH/QUICK,0x1bd06b96dd42ada85fdd0795f3b4a79db914add5,0.100856,9.915146,1.185710e+03,1.175649e+04,2368.884527,9.696290e+06,2368.884527,2407519,6.431892e+08,6.431892e+08,18,18,0x7ceb23fd6bc0add59e62ac25578270cff1b9f619,0x831753dd7087cac61ab5644b308642cc1c33dc13
3,USDC/GHST,0x096c5ccb33cfc5732bcd1f3195c13dbefc4c82f4,1.862181,0.537005,2.939019e+06,1.578267e+06,1434.978472,5.873631e+06,1436.055085,366287,2.365533e+08,2.365754e+08,6,18,0x2791bca1f2de4661ed88a30c99a7a9449aa84174,0x385eeac5cb85a38a9a07a70c73e0a3271cfb54a7
4,USDC/QUICK,0x1f1e4c845183ef6d50e9609f16f6f9cae43bc9cb,411.477804,0.002430,2.376786e+06,5.776220e+03,1161.988489,4.756237e+06,1161.988489,2138069,1.080357e+09,1.080357e+09,6,18,0x2791bca1f2de4661ed88a30c99a7a9449aa84174,0x831753dd7087cac61ab5644b308642cc1c33dc13
5,NEXO/WETH,0x10062ec62c0be26cc9e2f50a1cf784a89ded075f,1535.895846,0.000651,6.719878e+05,4.375218e+02,875.036140,3.596155e+06,875.043503,37315,9.619786e+06,9.617535e+06,18,18,0x41b3966b4ff7b427969ddf5da3627d6aeae9a48e,0x7ceb23fd6bc0add59e62ac25578270cff1b9f619
6,GFARM2/DAI,0x0c7ad41d3e0dbc1cfdcdd717afb0a72a65cdf069,0.001519,658.543895,1.386127e+03,9.128256e+05,444.135840,1.817927e+06,446.120418,39728,3.250942e+07,3.246414e+07,18,18,0x7075cab6bcca06613e2d071bd918d1a0241379e2,0x8f3cf7ad23cd3cadbd9735aff958023239c6a063
7,UCO/WETH,0x25bae75f6760ac30554cc62f9282307c3038c3a0,15356.989292,0.000065,2.092280e+06,1.362429e+02,272.485755,1.115669e+06,272.485755,485,1.993423e+05,1.985338e+05,18,18,0x3c720206bfacb2d16fa3ac0ed87d2048dbc401fc,0x7ceb23fd6bc0add59e62ac25578270cff1b9f619
8,USDC/MASK,0x253d637068fbf11b18d0f2a1bf3b167d37802687,9.758156,0.102478,5.300326e+05,5.431688e+04,257.867242,1.059602e+06,257.979865,16172,5.740340e+06,5.739340e+06,6,18,0x2791bca1f2de4661ed88a30c99a7a9449aa84174,0x2b9e7ccdf0f4e5b24757c1e1a80e311e34cb10c7
9,TECH/WETH,0x204a7adc76db7fe8c5e5f499cb3c4cff6d7282c2,66991.795779,0.000015,8.585178e+06,1.281527e+02,256.380123,1.053657e+06,256.305365,19357,4.203815e+11,5.941539e+06,18,18,0x6286a9e6f7e745a6d884561d88f94542d6715698,0x7ceb23fd6bc0add59e62ac25578270cff1b9f619


In [2]:
df.tail()

,symbols,pairAddr,price0,price1,reserve0,reserve1,reserveETH,reserveUSD,trackedReserveETH,txCount,untrackedVolumeUSD,volumeUSD,decimal0,decimal1,address0,address1
2995,WMATIC/POLYM,0x060a1fc8efaec79c5a49864e77b4e874ece42a03,0.000000,0.000000,0.000000,0.000000e+00,0.0,0.0,0.0,0,0.0,0.0,18,9,0x0d500b1d8e8ef31e21c99d1db9a6444d3adf1270,0xba53161d652f10c542dbfea50bb6545278c8fd51
2996,WMATIC/polyGAS,0x14176bd3666bf5a546ec0c7f7a200d6c10796ce1,0.000000,0.000000,0.000000,0.000000e+00,0.0,0.0,0.0,0,0.0,0.0,18,18,0x0d500b1d8e8ef31e21c99d1db9a6444d3adf1270,0x4a63e2e882a436d01b0fbc252dc081785c52c28c
2997,WMATIC/BOX,0x1e18a020748a2e4f93713e80e9b1ef872603e577,0.000000,0.000000,0.000000,0.000000e+00,0.0,0.0,0.0,0,0.0,0.0,18,18,0x0d500b1d8e8ef31e21c99d1db9a6444d3adf1270,0x94f749e6fc67dc8718fb0b2799fbe2958c79a7d0
2998,D/A,0x0602845b519e783afa6eb3b143d5e9f76734aaca,0.199222,5.019527,4480.538628,2.249018e+04,0.0,0.0,0.0,21,0.0,0.0,6,18,0x2f9cb2bc064c9f816d9fd481fedf545f2cb4d31d,0xfb341688ebf3dda9f95c2b916d98d47f47f95a44
2999,WMATIC/PGRA,0x1b214278435a08156f760775cbb777e9184da5cb,0.002012,496.911111,0.000000,2.230000e-14,0.0,0.0,0.0,2,0.0,0.0,18,18,0x0d500b1d8e8ef31e21c99d1db9a6444d3adf1270,0xa912914ef0d60db775d23ca31dfd10dc5bbe56c2
